<a href="https://colab.research.google.com/github/IAT-ComputationalCreativity-Spring2025/Week3-Rule-Based-Systems/blob/main/markov_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Markov Models Text Generation

This notebook introduces Markov chains for text generation. We'll build a simple
text generator that learns patterns from input text and generates new text with
similar statistical properties.

In [2]:
# First, let's import our required libraries
from collections import defaultdict
import random

## Building the Markov Chain

A Markov chain represents sequences of states where the probability of each state
depends only on the previous state(s). In our case, each state will be a sequence
of words, and we'll predict the next word based on this sequence.

In [3]:
def build_markov_chain(text, order=2):
    """
    Build a Markov chain from input text.

    Args:
        text (str): Input text to learn from
        order (int): Number of words to use as state (context)

    Returns:
        dict: Mapping from state tuples to lists of possible next words
    """
    chain = defaultdict(list)
    words = text.split()

    for i in range(len(words) - order):
        # Create state tuple from current words
        state = tuple(words[i:i+order])
        # Get the next word
        next_word = words[i+order]
        # Add to chain
        chain[state].append(next_word)

    return chain

## Generating Text

Now we'll use our Markov chain to generate new text. We'll randomly select from
the possible next words at each step.

In [4]:
def generate_text(chain, num_words=50):
    """
    Generate new text using the Markov chain.

    Args:
        chain (dict): Markov chain mapping states to possible next words
        order (int): Length of state tuples
        num_words (int): Number of words to generate

    Returns:
        str: Generated text
    """
    # Start with a random state from the chain
    words = list(random.choice(list(chain.keys())))
    order = len(list(chain.keys())[0])

    for _ in range(num_words - order):
        state = tuple(words[-order:])
        if state in chain:
            next_word = random.choice(chain[state])
            words.append(next_word)
        else:
            break

    return ' '.join(words)

## Part 3: Basic Example

Let's try our text generator with a simple example.

In [5]:
# Sample text
sample_text = """
The quick brown fox jumps over the lazy dog.
A quick brown dog jumps over the lazy fox.
The lazy fox sleeps while the quick brown dog watches.
"""

# Build the chain
chain = build_markov_chain(sample_text)

# Examine the chain
for state, words in chain.items():
    print(f"{' '.join(state)} -> {words}")

The quick -> ['brown']
quick brown -> ['fox', 'dog', 'dog']
brown fox -> ['jumps']
fox jumps -> ['over']
jumps over -> ['the', 'the']
over the -> ['lazy', 'lazy']
the lazy -> ['dog.', 'fox.']
lazy dog. -> ['A']
dog. A -> ['quick']
A quick -> ['brown']
brown dog -> ['jumps', 'watches.']
dog jumps -> ['over']
lazy fox. -> ['The']
fox. The -> ['lazy']
The lazy -> ['fox']
lazy fox -> ['sleeps']
fox sleeps -> ['while']
sleeps while -> ['the']
while the -> ['quick']
the quick -> ['brown']


In [6]:
# Generate some text
print("Generated text:")
print(generate_text(chain))

Generated text:
brown fox jumps over the lazy dog. A quick brown dog jumps over the lazy fox. The lazy fox sleeps while the quick brown dog jumps over the lazy fox. The lazy fox sleeps while the quick brown dog jumps over the lazy dog. A quick brown fox jumps over


## Student Tasks

1. Basic Implementation:
   - Try changing the order parameter in build_markov_chain
   - What happens with order=1 vs order=3?

In [7]:
# Task 1: Experiment with different orders
print("\nOrder 1:")
chain1 = build_markov_chain(sample_text, order=1)
print(generate_text(chain1))

print("\nOrder 3:")
chain3 = build_markov_chain(sample_text, order=3)
print(generate_text(chain3))


Order 1:
the quick brown dog jumps over the lazy fox jumps over the quick brown fox jumps over the quick brown fox jumps over the quick brown dog watches.

Order 3:
brown fox jumps over the lazy fox. The lazy fox sleeps while the quick brown dog jumps over the lazy fox. The lazy fox sleeps while the quick brown dog jumps over the lazy fox. The lazy fox sleeps while the quick brown dog watches.


2. Use Your Own Text:
   Below, try using a different text source. You could use:
   - Song lyrics
   - Book excerpts
   - Movie quotes

In [8]:
# Task 2: Add your own text here
your_text = """
Then the hobbit slipped on his ring, and warned by the echoes to take more than hobbit's care to make no sound, he crept noiselessly down, down, down into the dark.
He was trembling with fear, but his little face was set and grim. Already he was a very different hobbit from the one that had run out without a pocket-handkerchief from Bag-End long ago.
He had not had a pocket-handkerchief for ages. He loosened his dagger in its sheath, tightened his belt, and went on. “Now you are in for it at last, Bilbo Baggins,” he said to himself.
“You went and put your foot right in it that night of the party, and now you have got to pull it out and pay for it! Dear me, what a fool I was and am!” said the least Tookish part of him.
“I have absolutely no use for dragon-guarded treasures, and the whole lot could stay here for ever, if only I could wake up and find this beastly tunnel was my own front-hall at home!”
He did not wake up of course, but went still on and on, till all sign of the door behind had faded away. He was altogether alone. Soon he thought it was beginning to feel warm.
“Is that a kind of a glow I seem to see coming right ahead down there?” he thought.  It was. As he went forward it grew and grew, till there was no doubt about it.
It was a red light steadily getting redder and redder. Also it was now undoubtedly hot in the tunnel. Wisps of vapour floated up and past him and he began to sweat.
A sound, too, began to throb in his ears, a sort of bubbling like the noise of a large pot galloping on the fire, mixed with a rumble as of a gigantic tom-cat purring.
This grew to the unmistakable gurgling noise of some vast animal snoring in its sleep down there in the red glow in front of him.   It was at this point that Bilbo stopped.
Going on from there was the bravest thing he ever did. The tremendous things that happened afterward were as nothing compared to it.
He fought the real battle in the tunnel alone, before he ever saw the vast danger that lay in wait.
At any rate after a short halt go on he did; and you can picture him coming to the end of the tunnel, an opening of much the same size and shape as the door above.
Through it peeps the hobbit's little head. Before him lies the great bottommost cellar or dungeon-hall of the ancient dwarves right at the Mountain's root.
It is almost dark so that its vastness can only be dimly guessed, but rising from the near side of the rocky floor there is a great glow. The glow of Smaug!

"""

print("\nOrder 1:")
mychain1 = build_markov_chain(your_text, order=1)
print(generate_text(mychain1))

print("\nOrder 3:")
mychain3 = build_markov_chain(your_text, order=3)
print(generate_text(mychain3))



Order 1:
point that night of bubbling like the same size and redder. Also it out without a gigantic tom-cat purring. This grew to the tunnel. Wisps of the party, and the same size and past him coming to throb in the tunnel, an opening of much the bravest thing he ever

Order 3:
no doubt about it. It was a red light steadily getting redder and redder. Also it was now undoubtedly hot in the tunnel. Wisps of vapour floated up and past him and he began to sweat. A sound, too, began to throb in his ears, a sort of bubbling like


3. Advanced Implementation:
   Add temperature-based sampling to control randomness

In [18]:
def generate_text_with_temperature(chain, temperature, num_words=30, order=1):
    """
    Generate text with temperature-based sampling.
    Lower temperature = more conservative/predictable
    Higher temperature = more random/creative

    Args:
        chain (dict): Markov chain
        temperature (float): Controls randomness (0.1 to 2.0 recommended)
        order (int): Length of state tuples
        num_words (int): Number of words to generate
    """
    words = list(random.choice(list(chain.keys())))
    order = len(list(chain.keys())[0])

    for _ in range(num_words - order):
        state = tuple(words[-order:])
        if state in chain:
            # Count frequencies of next words
            next_words = chain[state]
            word_counts = defaultdict(int)
            for word in next_words:
                word_counts[word] += 1

            # Apply temperature
            weights = [count ** (1.0 / temperature) for count in word_counts.values()]
            total = sum(weights)
            weights = [w/total for w in weights]

            # Choose next word based on weighted probabilities
            next_word = random.choices(list(word_counts.keys()), weights=weights)[0]
            words.append(next_word)
        else:
            break

    return ' '.join(words)

# Try different temperatures
# print("\nLow temperature (more predictable):")
# print(generate_text_with_temperature(chain, temperature=0.3))

# print("\nHigh temperature (more random):")
# print(generate_text_with_temperature(chain, temperature=2.0))

# print("\nLow temperature (more predictable):")
# print(generate_text_with_temperature(mychain1, temperature=0.3))

# print("\nHigh temperature (more random):")
# print(generate_text_with_temperature(mychain1, temperature=2.0))

# print("\nLow temperature (more predictable):")
# print(generate_text_with_temperature(mychain3, temperature=0.3))

chain4 = build_markov_chain(your_text, order = 1)

print("\ndefault:")
print(generate_text(chain4))
print("\ntemp 2 order 3:")
print(generate_text_with_temperature(chain4, temperature=2.0, order =3.0))
print("\ntemp 2 order 10:")
print(generate_text_with_temperature(chain4, temperature=2.0, order =10.0))
print("\ntemp 4 order 1:")
print(generate_text_with_temperature(chain4, temperature=4.0, order =1))


default:
me, what a very different hobbit slipped on his ears, a red light steadily getting redder and am!” said the tunnel. Wisps of him. It was trembling with a kind of vapour floated up of Smaug!

temp 2 order 3:
Mountain's root. It is a pocket-handkerchief for ever, if only I seem to pull it peeps the great bottommost cellar or dungeon-hall of the hobbit from Bag-End long ago. He

temp 2 order 10:
real battle in its vastness can only be dimly guessed, but his dagger in its sleep down there was the same size and redder. Also it was trembling with fear,

temp 4 order 1:
sweat. A sound, too, began to the near side of some vast danger that Bilbo stopped. Going on his dagger in its sheath, tightened his belt, and past him and


## Challenge Tasks:

1. Implement a function to analyze the Markov chain:
   - Count the number of unique states
   - Find the most common transitions
   - Calculate the average number of possible next words for each state

In [33]:
def analyze_chain(chain):
    """
    Analyze properties of the Markov chain.

    Args:
        chain (dict): Markov chain to analyze
    """
    num_states = len(chain)
    total_transitions = sum(len(next_words) for next_words in chain.values())
    avg_transitions = total_transitions / num_states if num_states > 0 else 0

    # Find most common next word for each state
    most_common = {}
    for state, next_words in chain.items():
        word_counts = defaultdict(int)
        for word in next_words:
            word_counts[word] += 1
        most_common[state] = max(word_counts.items(), key=lambda x: x[1])

    print(f"Number of unique states: {num_states}")
    print(f"Average transitions per state: {avg_transitions:.2f}")
    print("\nMost common transitions:")
    for state, (word, count) in list(most_common.items())[:5]:  # Show top 5
        print(f"{' '.join(state)} -> {word} (count: {count})")

# Analyze our chain
print("\nChain Analysis:")
analyze_chain(chain)

analyze_chain(mychain1)


analyze_chain(mychain3)


Chain Analysis:
Number of unique states: 20
Average transitions per state: 1.30

Most common transitions:
The quick -> brown (count: 1)
quick brown -> dog (count: 2)
brown fox -> jumps (count: 1)
fox jumps -> over (count: 1)
jumps over -> the (count: 2)
Number of unique states: 273
Average transitions per state: 1.77

Most common transitions:
Then -> the (count: 1)
the -> door (count: 2)
hobbit -> slipped (count: 1)
slipped -> on (count: 1)
on -> his (count: 1)
Number of unique states: 480
Average transitions per state: 1.00

Most common transitions:
Then the hobbit -> slipped (count: 1)
the hobbit slipped -> on (count: 1)
hobbit slipped on -> his (count: 1)
slipped on his -> ring, (count: 1)
on his ring, -> and (count: 1)


## Further Exploration:

Other ideas to try:
1. Modify the code to preserve punctuation
2. Add start-of-sentence and end-of-sentence tokens
3. Implement bi-directional generation
4. Create a chain that works with characters instead of words
5. Add input validation and error handling